In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors
import cv2

In [ ]:
def show_image_plt(img_rgb):
    #rgb_carrot = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.show()

def show_image(img):
    cv2.imshow("Image", img)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

In [ ]:
def resize_to_square(vegi):
    img_size = 256
    height, width = vegi.shape[:2]
    a1 = width / height
    a2 = height / width

    if (a1 > a2):
        r_img = cv2.resize(vegi, (round(img_size * a1), img_size), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[1]/6)
        crop_img = r_img[0:img_size, margin:(margin+img_size)]

    elif(a1 < a2):
        # if height greater than width
        r_img = cv2.resize(vegi, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        margin = int(r_img.shape[0]/6)
        crop_img = r_img[margin:(margin+img_size), 0:img_size]

    elif(a1 == a2):
        # if height and width are equal
        r_img = cv2.resize(vegi, (img_size, round(img_size * a2)), interpolation = cv2.INTER_AREA)
        crop_img = r_img[0:img_size, 0:img_size]

    if(crop_img.shape[0] != img_size or crop_img.shape[1] != img_size):
        crop_img = r_img[0:img_size, 0:img_size]

    return crop_img

In [ ]:
#path = "G:\Meine Ablage\KI_Projekt\Bilder\Zwiebel\zwiebel_9.jpg"
#vegi = cv2.imread(path.replace("\\","/")) # As BGR

def draw_contours(bgr_img, object_area=500):
    img_256 = resize_to_square(bgr_img)
    #gray = cv2.cvtColor(img_256, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(img_256, cv2.COLOR_BGR2HSV)

    hue, saturation, value = cv2.split(hsv)

    blurred_sat = cv2.GaussianBlur(saturation, (5, 5), 0)

    # Thresh 50 - 100
    # Or from 85 - 100 removes some more small edges
    # Thresh from 120 only shows the edge from vegi: karotte_107.jpg"
    # Compute the thresh dynamically from the mean() value. 
    thresh = blurred_sat.mean()
    # The factors were simply selected by testing the algoritm. Another approach could be to calculate the mean with the standard deviation -> Test it ???
    thresh_low = 0.3 * thresh 
    thresh_high = 2 * thresh
    #print(f"Thresh low: {thresh_low} and Thresh high: {thresh_high}")
    # The next four lines control how good the bounding box will fit
    edges = cv2.Canny(blurred_sat, thresh_low, thresh_high)

    kernel = np.ones((4, 4), np.uint8) # creates 4x4 Identity matrix

    # To see the effect if changing kernel and iterations plot it. It seems if the value is to small than the probabillity is higher that the edge lines are not closed
    dilate = cv2.dilate(edges, kernel, iterations=6) 
    #show_image_plt(dilate)

    erode = cv2.erode(dilate, kernel, iterations=4)
    #show_image_plt(erode)

    contours, hierarchy = cv2.findContours(erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10] # ???
   
    img_256_copy = img_256.copy()
    # Flag makes sure that there is a maximum of 1 box in each image. Assumption, the bounding box for the vegi is always the biggest
    more_than_one_box = False  
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area >= object_area:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(img_256_copy, [box], 0, (0, 255, 0), 2)
            #print("Draw Contour")
            if i > 0:
                # There are more than 2 boxes in the image
                more_than_one_box = True

    rgb = cv2.cvtColor(img_256_copy, cv2.COLOR_BGR2RGB)
    return rgb, more_than_one_box


Execute the cell below to load the images from the Bilder folder, run them through the draw_contours function and save them into the BildermitBounding folder.
Uncomment the last line.

In [ ]:
#image_folders = ['Karotte', 'Zwiebel', 'Kartoffel']
image_folders = ['Karotte_Trieb']
for fold in image_folders:
    print(f"Edit Images in: {fold}\n")
    # Get all images in Folder
    for i, fname in enumerate(os.listdir(f"G:/Meine Ablage/KI_Projekt/Bilder/{fold}")):
        if i == 150:
            # At the moment draw only boxes for the first 150 images
            break

        path = f"G:\Meine Ablage\KI_Projekt\Bilder\{fold}\{fname}"
        vegi = cv2.imread(path.replace("\\","/"))

        rgb, more_than_one_box = draw_contours(vegi)
        
        if more_than_one_box:
            box_size = 500
            while more_than_one_box:
                box_size += 250
                rgb, more_than_one_box = draw_contours(vegi, box_size)

            print("************* Increased Box Size by 250 *****************")
    
        cv2.imwrite(f"G:\Meine Ablage\KI_Projekt\Bilder\BildermitBounding\{fold}_Bounding_Leon_2\{fname}", cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB))